# Неперсонализованные рекомендации (Non-Personalised Recommendations)

Когда пользователь только приходит на сервис мы ничего не знаем о его вкусах, так как откликов от него еще не было. В таком случае применяют неперсонализованные рекомендации.

### Популярные рекомендации

Первая мысль которая может придти какой товар порекомендовать - это порекомендовать популярный товар. Но тогда встает вопрос - что значит популярный?
Это может значить например:
<ul>
    <li>наиболее просматриваемый</li>
    <li>наиболее кликабельный</li>
    <li>наиболее покупаемый</li>
    <li>товары которые берут чаще всего с текущими в вашей корзине покупок</li>
</ul>
Выбор определения зависит от конкретного сервиса и каких целей вы приследуете 

In [1]:
import pandas as pd

In [2]:
df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv', sep=',')

In [3]:
df_movies = pd.read_csv('data/ml-latest-small/movies.csv')

In [4]:
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [5]:
df_ratings['timestamp'].max()

1537799250

In [6]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'], unit='s').sort_values()

In [21]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        #выдаем фильмы которым чаще всего ставят оценку
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column]\
                                 .value_counts()\
                                 .head(self.max_K)\
                                 .index.values
    
    def recommend(self,  N=10):
        return self.recommendations[:N]

In [22]:
recommender = PopularRecommender(days=11, item_column='movieId', dt_column='timestamp')

In [23]:
recommender.fit(df_ratings)

In [25]:
recommender.recommend()

array([122906, 187593,  35836, 148626,  88140,  81845, 187595, 164179,
       177765, 106920])

In [26]:
pd.DataFrame(recommender.recommend(), columns=['movieId']).merge(df_movies)

,movieId,title,genres
0,122906,Black Panther (2017),Action|Adventure|Sci-Fi
1,187593,Deadpool 2 (2018),Action|Comedy|Sci-Fi
2,35836,"40-Year-Old Virgin, The (2005)",Comedy|Romance
3,148626,"Big Short, The (2015)",Drama
4,88140,Captain America: The First Avenger (2011),Action|Adventure|Sci-Fi|Thriller|War
5,81845,"King's Speech, The (2010)",Drama
6,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi
7,164179,Arrival (2016),Sci-Fi
8,177765,Coco (2017),Adventure|Animation|Children
9,106920,Her (2013),Drama|Romance|Sci-Fi


## Демографические группы, сегменты популярных рекоммендаций 

Однако легко может случится так, что популярные продукты не соответствуют конкретно вашим вкусам. Но вы можете принадлежать некоторой демографической группе, что может хорошо определить ваши вкусы.

Если в начале у нас имеется информация о самом пользователе, такие как:
<ul>
    <li>Пол</li>
    <li>Возраст</li>
    <li>Семейный статус</li>
    <li>Социальный статус</li>
    <li>Вид и область занятости</li>
    <li>Местоположение</li>
    <li>итд</li>
</ul>
Можно провести аналитики на отдельных группах и подобрать более специфичную

### Всегда ли полезны популярные рекоммендации?

Нет, не всегда. Если подумать, то ваши вкусы вообще говоря могут и не совпадать с большинством. Например вам может нравиться старый рок, а от новых рэп альбомов вы не в восторге. Другим недостатком, когда популярные товары будут плохой рекоммендацией - их очевидность. Если вы порекомендуете популярные товары в супермаркете, то скорее всего это будут бананы, молоко, хлеб, пакет, гречка итд. Иными словами что-то что вы бы и так купили, а рекомендация полностью бесполезна и ничего нового вам не предложила

### Assosiation rules

Другим простым, но эффективным способом порекомендовать товары - это посмотреть какие товары чаще встречаются в одной корзине (транзакции, последовательности действий). В данном случае мы пытаемся определить есть ли взаимосвязь в имеющихся данных продуктов. Рекомендации такого вида можно описать как "С этими товарами также берут", "Те кто купил X, также купили Y"

<img src="img/cust_who_bought.png" alt="Сustomers Who bought" style="width: 600px;"/>

Для начала введем несколько определений которые будут описывать эти правила

Пусть T - множество транзакций, X, Y - подмножестов товаров

In [27]:
df = pd.DataFrame({
    'transaction_id': list(range(5)),
    'milk': [1, 0, 0, 1, 0],
    'bread': [1, 0, 0, 1, 1],
    'butter': [0, 1, 0, 1, 0],
    'beer': [0, 0, 1, 0, 0],
    'diapers': [0, 0, 1, 0, 0]})

In [28]:
df.head()

,transaction_id,milk,bread,butter,beer,diapers
0,0,1,1,0,0,0
1,1,0,0,1,0,0
2,2,0,0,0,1,1
3,3,1,1,1,0,0
4,4,0,1,0,0,0


### Support
$$ supp(X) = \frac{|\{X\subset{T}\}|}{|T|} abc$$
Support - частотность транзакций с подмножестовм товаров ко всем транзакциям. Стоит отметить, что нотация $supp(x_1\cup{x_2})$ будет означать частотность транзакций, когда товары $x_1$ и $x_2$ встречаются транзакции вместе

In [14]:
def support(df, ascendents):
    pass

assert support(df, ['milk']) == 0.4
assert support(df, ['butter', 'bread']) == 0.2

### Confidence
$$ conf(X->Y)=\frac{supp(X\cup{Y})}{supp(X)}$$
Confidence (достоверность) - показатель того, как часто правило X -> Y срабатывает для всего датасета. Или более математическим способом P(Y | X)

In [15]:
ascendent = X
conscendent = Y
def confidence(df, ascendents, conscendents):
    # code here
    pass

assert confidence(df, ['bread', 'butter'], ['milk']) == 1 # правило корректно в 100% случаев
assert confidence(df, ['butter'], ['bread']) == 0.5 # правило срабатывает в 50% случаев

### Lift
$$Lift(X->Y)=\frac{supp(X\cup{Y})}{supp(X)*supp(Y)}$$
Lift (поддержка) - показатель того, что X и Y являются независимыми наборами товаров.
* Если lift = 1 утверждается, что появление товаров независимо и собственно правила здесь нет
* Если lift < 1, это позволяет нам понять, что элементы заменяют друг друга. Это означает, что наличие одного элемента отрицательно влияет на наличие другого элемента и наоборот.
* Если lift > 1, то рассматриваемое правило потенциально полезно и чем lift выше, тем "сильнее правило"

In [16]:
def lift(df, ascendents, conscencdents):
    # code here
    pass

assert lift(df, ['milk', 'bread'], ['butter']) == 1.25

На практике нам нужно найти сами правила которые удовлетворяют заданным порогам по support и lift. Для этого есть несколько алгоритмов: *BruteForce*, *Apriori algorythm*, *ECLAT Algorithm* и *FP-Growth Algorithm*. Для примера мы используем реализацию алгоритма apriory из библиотеки mlxtend.

<img src="img/apriory.png" alt="Сustomers Who bought" style="width: 600px;"/>

Если коротко про алгоритм, то он строит дерево перебора правил и рассчитывает support для них. Если посчитанный support для правила не проходит по порогу, то любое супермножество этого правило далее не рассматриваются

Подробнее о метриках выше и алгоритмах вы можете почитать здесь
https://habr.com/ru/company/ods/blog/353502/#4

In [17]:
# !pip install mlxtend  

In [29]:
from mlxtend.frequent_patterns import (apriori, association_rules)

In [30]:
df = pd.read_csv('data/test.csv')

In [31]:
df.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [32]:
df['Item'].value_counts()

Coffee            5471
Bread             3325
Tea               1435
Cake              1025
Pastry             856
                  ... 
Raw bars             1
Polenta              1
Olum & polenta       1
The BART             1
Gift voucher         1
Name: Item, Length: 95, dtype: int64

In [33]:
# убираем повторения и пропущенные значения
df = df.sort_values(['Date', 'Time', 'Transaction'])
df.drop_duplicates(inplace=True)
df = df[df['Item'] != "NONE"]

In [35]:
df.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies


In [36]:
df['Transaction'].nunique()

9465

In [37]:
df['count'] = 1
df_transactions = df.groupby(['Transaction', 'Item'])['count'].sum().unstack().fillna(0)

In [38]:
freq_items = apriori(df_transactions, min_support=0.01, use_colnames=True)

In [39]:
freq_items.sort_values('support', ascending=False)

,support,itemsets
6,0.478394,(Coffee)
2,0.327205,(Bread)
26,0.142631,(Tea)
4,0.103856,(Cake)
34,0.090016,"(Bread, Coffee)"
...,...,...
11,0.010565,(Hearty & Seasonal)
20,0.010460,(Salad)
30,0.010354,"(Alfajores, Bread)"
58,0.010037,"(Cake, Bread, Coffee)"


In [40]:
rules = association_rules(freq_items, metric='lift', min_threshold=1)

In [41]:
rules.sort_values('lift', ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
40,(Cake),"(Tea, Coffee)",0.103856,0.049868,0.010037,0.096643,1.937977,0.004858,1.051779
39,"(Tea, Coffee)",(Cake),0.049868,0.103856,0.010037,0.201271,1.937977,0.004858,1.121962
8,(Cake),(Hot chocolate),0.103856,0.058320,0.011410,0.109868,1.883874,0.005354,1.057910
9,(Hot chocolate),(Cake),0.058320,0.103856,0.011410,0.195652,1.883874,0.005354,1.114125
11,(Tea),(Cake),0.142631,0.103856,0.023772,0.166667,1.604781,0.008959,1.075372
